In [1]:
from langchain import OpenAI, PromptTemplate, LLMChain
from dotenv import load_dotenv
from pandas import DataFrame
load_dotenv();

In [2]:
template = """{variable} is a(n) {term} iff {definition}.

Using the above definition, is {individual} a(n) {term}? If not, why not?"""

In [3]:
prompt = PromptTemplate(
    input_variables=["variable", "term", "definition", "individual"], 
    template=template,
)

In [4]:
haslanger_woman = """(i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination"""

In [5]:
print(prompt.format(variable="S", term="woman", definition=haslanger_woman, individual="Rosa Parks"))

S is a(n) woman iff (i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination.

Using the above definition, is Rosa Parks a(n) woman? If not, why not?


In [6]:
llm = OpenAI(temperature=0.1)
chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
humans = [
    "Rosa Parks",
    "Hillary Clinton",
    "Pamela Anderson",
    "Lebron James",
    "Sally Haslanger",
    "Nancy Pelosi",
    "Catwoman",
    "Taylor Swift",
    "Julia Roberts",
    "James Earl Carter",
    "Peter Bogdanovich",
    "Elinor Ostrom",
    "Catherine of Aragon",
    "Catherine the Great",
    "Joyce Carol Oates",
    "Sally Hemings",
    "Caitlyn Jenner",
    "Kamala Harris"
]

In [8]:
df = DataFrame(
    [
        [
            individual, 
            chain.run(variable="S", term="woman", definition=haslanger_woman, individual=individual).strip() 
        ] 
        for individual in humans 
    ],
    columns = ['individual' , 'reason']
)

In [9]:
df.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})

,individual,reason
0,Rosa Parks,"No, Rosa Parks is not a woman. Although she was regularly and for the most part observed or imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction, her social position was not subordinate and she did not occupy a position of systematic subordination. Therefore, she does not satisfy the criteria for being a woman as defined above."
1,Hillary Clinton,"Yes, Hillary Clinton is a woman. She has the bodily features presumed to be evidence of a female’s biological role in reproduction, and her gender marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. Furthermore, the fact that she satisfies these criteria plays a role in her systematic subordination, as her gender has been used to justify her social position and her systematic subordination."
2,Pamela Anderson,"Yes, Pamela Anderson is a woman. She has the bodily features presumed to be evidence of a female’s biological role in reproduction, and her social position is subordinate to that of men in her society. Furthermore, her gender plays a role in her systematic subordination, as she is often objectified and sexualized in the media."
3,Lebron James,"No, Lebron James is not a woman. Lebron James does not have the bodily features presumed to be evidence of a female's biological role in reproduction, and thus does not satisfy the first criterion of the definition. Additionally, Lebron James does not occupy a subordinate social position that is motivated and justified by his bodily features, and thus does not satisfy the second criterion of the definition. Therefore, Lebron James does not satisfy the definition of a woman."
4,Sally Haslanger,"Yes, Sally Haslanger is a woman. She has the bodily features presumed to be evidence of a female’s biological role in reproduction, and her social position is subordinate in her society. Furthermore, her satisfying the criteria of (i) and (ii) plays a role in her systematic subordination."
5,Nancy Pelosi,"Yes, Nancy Pelosi is a woman. She has the bodily features presumed to be evidence of a female’s biological role in reproduction, and her gender marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. Furthermore, the fact that she satisfies these criteria plays a role in her systematic subordination, as her gender has been used to justify her occupying a subordinate social position."
6,Catwoman,"No, Catwoman is not a woman according to the definition provided. Catwoman is a fictional character and does not have the bodily features or occupy the social position of a woman. Therefore, she does not satisfy the criteria of (i) and (ii) and does not experience systematic subordination."
7,Taylor Swift,"Yes, Taylor Swift is a woman. She has the bodily features presumed to be evidence of a female’s biological role in reproduction, and her gender marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. Furthermore, the fact that she satisfies these criteria plays a role in her systematic subordination, as her gender has been used to oppress her in various ways."
8,Julia Roberts,"Yes, Julia Roberts is a woman. She has the bodily features presumed to be evidence of a female's biological role in reproduction, and her gender marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social positions that are in fact subordinate. Furthermore, the fact that she satisfies these criteria plays a role in her systematic subordination."
9,James Earl Carter,"No, James Earl Carter is not a woman. While he may have certain bodily features that are associated with women, he does not satisfy the other two criteria of the definition. Specifically, he does not occupy a